In [2]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read Category:Prince (プリンス）articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22392993)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (プリンス|2　生年|20　人物|20)

In [3]:
df = pd.read_csv("prince.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,リチャード2世_(イングランド王),30441,NaN,30788,20210718113636
1,2,エドワード5世_(イングランド王),30504,NaN,7117,20210225064849
2,3,ヘンリー5世_(イングランド王),30523,NaN,26071,20210823104631
3,4,エドワード黒太子,33577,NaN,16332,20220513192154
4,5,ヘンリー8世_(イングランド王),34653,NaN,50539,20220321105242
5,6,エドワード6世_(イングランド王),34949,NaN,6622,20210925190316
6,7,ジョージ2世_(イギリス王),47495,NaN,69607,20220608174625
7,8,フランソワ1世_(フランス王),52852,NaN,19736,20220105040706
8,9,ルイ12世_(フランス王),52854,NaN,9307,20211203130636
9,10,ルイ・フィリップ_(フランス王),85516,NaN,17373,20220127064432


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     334 non-null    int64  
 1   title      334 non-null    object 
 2   pageid     334 non-null    int64  
 3   namespace  0 non-null      float64
 4   length     334 non-null    int64  
 5   touched    334 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 15.8+ KB


In [8]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info

# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [9]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return
        
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [10]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()
    
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

### CHANGE HERE


In [11]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/prince.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/prince/'

In [12]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [13]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [14]:
not_found = {}      # dict for not-found data

In [15]:
# measure time
start_time = time.time()

In [16]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
1 リチャード2世 (イングランド王)
processing_time(sec):  2.26277756690979
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
2 エドワード5世 (イングランド王)
processing_time(sec):  3.318588972091675
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
3 ヘンリー5世 (イングランド王)
processing_time(sec):  4.460619211196899
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
4 エドワード黒太子
processing_time(sec):  5.693782567977905
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
5 ヘンリー8世 (イングランド王)
processing_time(sec):  7.116521120071411
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AB
6 エドワード6世 (イングランド王)
processing_time(sec):  8.508952379226685
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
7 ジョージ2世 (イギリス王)
processing_time(sec):  8.952762603759766
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
8 フランソワ1世 (フランス王)
processing_time(sec):  9.675391912460327
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
9 ルイ12世 (フランス王)
processin

70 フェルディナント・フォン・プロイセン
processing_time(sec):  120.94698739051819
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
71 アウグスト・フォン・プロイセン
processing_time(sec):  122.6771628856659
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
72 フーベルトゥス・フォン・プロイセン
processing_time(sec):  123.81276273727417
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
73 ルートヴィヒ・フォン・プロイセン
processing_time(sec):  124.834636926651
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
74 ゲオルク・フリードリヒ・フォン・プロイセン
processing_time(sec):  125.92167401313782
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
75 カール・エドゥアルト (ザクセン＝コーブルク＝ゴータ公)
processing_time(sec):  127.19734525680542
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
76 ゲオルク5世 (ハノーファー王)
processing_time(sec):  128.4911346435547
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
77 カロル1世 (ルーマニア王)
processing_time(sec):  129.98015594482422
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AK
78 ヨハン・ウィレム・フリーゾ (オラニエ公)
processing_time(se

137 フィリップ (パリ伯)
processing_time(sec):  221.45527839660645
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
138 ルイトポルト・フォン・バイエルン
processing_time(sec):  223.59700918197632
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
139 アンリ3世 (コンデ公)
processing_time(sec):  225.85993337631226
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
140 フィリップ2世 (オルレアン公)
processing_time(sec):  228.2558274269104
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
141 ルイ・ド・ブルボン＝オルレアン (オルレアン公)
processing_time(sec):  231.39220786094666
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
142 ルイ・フィリップ1世 (オルレアン公)
processing_time(sec):  233.47201371192932
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
143 ナポレオン・ヴィクトル・ボナパルト
processing_time(sec):  235.8862748146057
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
144 ルイ・ナポレオン (ナポレオン公)
processing_time(sec):  238.51146245002747
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
145 ルイ・アレクサンドル・ド・ブルボン (トゥールーズ伯)
processing_tim

206 クリストフォロス (ギリシャ王子)
processing_time(sec):  370.7898564338684
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
207 ハインリヒ・フォン・バイエルン
processing_time(sec):  371.2391173839569
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
208 ニコラエ・アル・ロムニエイ
processing_time(sec):  371.8046405315399
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
209 ユニオ・ヴァレリオ・ボルゲーゼ
processing_time(sec):  373.4847466945648
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
210 シャルル1世・ド・ブルボン (ルーアン大司教)
processing_time(sec):  375.72180008888245
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
211 シャルル・ド・ブルボン＝ソワソン
processing_time(sec):  377.86071372032166
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
212 シャルル4世 (アランソン公)
processing_time(sec):  379.8653733730316
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AT
213 アナトーリー・デミドフ
processing_time(sec):  382.9644842147827
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
214 ゲオルギー・ミハイロヴィチ・ロマノフ
processing_time(sec):  383.4885654

273 アレクサンダー・フェルディナント・フォン・プロイセン
processing_time(sec):  500.5051941871643
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
274 ベルトルト・フォン・バーデン
processing_time(sec):  500.8837447166443
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
275 フリードリヒ・ジギスムント・フォン・プロイセン
processing_time(sec):  501.6073272228241
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
276 マクシミリアン・アンドレアス・フォン・バーデン
processing_time(sec):  504.3826856613159
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
277 モーリッツ・フォン・デア・プファルツ
processing_time(sec):  507.2226507663727
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
278 フィリップ・フォン・デア・プファルツ (1627-1650)
processing_time(sec):  510.69665789604187
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
279 フェルディナント・フォン・バイエルン (1699-1738)
processing_time(sec):  514.485221862793
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
280 クロード・ルイ・エクトル・ド・ヴィラール
processing_time(sec):  516.8853974342346
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AW
